In [3]:
import numpy as np
import itertools
from architectures.sample_conv import ConvNetMNIST, ConvNetCIFAR
from data_eng.dataset_loader import load_MNIST, load_imagenette, load_CIFAR10
from torchvision import transforms
import torch
from data_eng.io import load_model
from evaluation.metrics import evaluate_attack, evaluate_model
from evaluation.visualization import simple_visualize
from attacks.pgd_mn import PGD_MN
import datetime
from tqdm import tqdm



In [4]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MODEL_SAVE_PATH = './models/cnn-mnist.pt'

model = load_model(ConvNetMNIST().to(device), MODEL_SAVE_PATH)

transform = transforms.Compose([
    transforms.ToTensor()
])

_, test_loader = load_MNIST(transform=transform, batch_size=1)


In [5]:
cnt = 0

for images, labels in test_loader:
    images = images.to(device)
    print(torch.min(images))
    print(torch.max(images))

    break

tensor(0., device='cuda:0')
tensor(1., device='cuda:0')


In [8]:
from attacks.white_box import PGD

atk = PGD(model, eps=8/255)
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    print(adv_images.shape)
    end = datetime.datetime.now()
    # acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(0, int((end-start).total_seconds()*1000)))
    break

torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
- Robust Acc: 0 (21 ms)


In [ ]:
att = PGD_MN(model)
attack_results = att.forward(test_loader)


In [4]:

attack_eval = evaluate_attack(attack_results, 10)

print(attack_eval)
print(attack_eval.conf_matrix)

Accuracy: 89.63, Precision: 91.05, Recall: 89.44, F1: 89.7
[[ 881    8    5    4   21    7    7    5   10   19]
 [   0 1133    0    0    0    0    0    1    0    0]
 [   2   44  888    8    6    0    1   12   65    0]
 [   0   35    6  817    2    3    0    0  109   20]
 [   1   26    3    0  926    0    1    2   12    0]
 [   2   17    1   18    1  726   11    4   84    6]
 [   2   28    5    1   39    4  856    0    9    1]
 [   2   32   15    1    5    0    0  927   14    6]
 [   3    8    5    0    6    2    3    2  926    3]
 [   3   10    2    2  121    2    0   26   70  743]]
